# Boilerplate

In [1]:
import sys
sys.path.insert(0, 'venv/lib/python3.10/site-packages')
import asyncio
import time

from nwv2_python_wrapper import *
import nwv2_python_wrapper

# Run

List all Python wrappers

In [2]:
[f for f in dir(nwv2_python_wrapper) if not f.startswith('__') and f != 'nwv2_python_wrapper']

['BroadcastChatMessageW',
 'EndpointW',
 'FilterCmdW',
 'FilterInterface',
 'FilterModeW',
 'FilterNoticeW',
 'FilterRspW',
 'GameOptionsW',
 'GameOutcomeW',
 'GameUpdateW',
 'GenPartInfoW',
 'GenStateDiffPartW',
 'GenStateDiffW',
 'NetRegionW',
 'PacketSettingsW',
 'PacketW',
 'PlayerInfoW',
 'ProcessUniqueIdW',
 'RequestActionW',
 'ResponseCodeW',
 'RoomListW',
 'TransportCmdW',
 'TransportInterface',
 'TransportNoticeW',
 'TransportRspW',
 'UniUpdateW',
 'new_transport_interface']

## Transport layer setup

Create the Transport layer and run it in the background.

In [3]:
tiface_inner = await new_transport_interface("0.0.0.0", 0)
tiface_inner.run()

<Future pending cb=[<builtins.PyDoneCallback object at 0x7fc88bf9c850>()]>

## Wrap the PYO3 Transport Interface in a Python class with the same methods

Note that there is no `run()`. Hence, the passed in `TransportInterface` instance must have already had its `run()` method called.

In [4]:
class LoggingTransportInterface:
    def __init__(self, inner):
        self.inner = inner
        self.ctr = 0
        self.skipped_count = None
        
    async def command_response(self, transport_cmd):
        print(f"LTI{self.ctr}{self.skipped()}: received transport_cmd {transport_cmd}")
        self.ctr += 1
        transport_rsp = await self.inner.command_response(transport_cmd)
        print(f"LTI{self.ctr}: got transport_rsp back from Transport layer: {transport_rsp}")
        self.ctr += 1
        return transport_rsp
    
    def get_notifications(self):
        transport_notif_list = self.inner.get_notifications()
        if self.skipped_count is None:
            print(f"LTI{self.ctr}: got transport notifications: {transport_notif_list}")
            self.skipped_count = 0
            if len(transport_notif_list) > 0:
                self.reset()
        else:
            self.skipped_count += 1
        self.ctr += 1
        return transport_notif_list
    
    def skipped(self):
        if self.skipped_count is None:
            return ""
        count = self.skipped_count
        self.skipped_count = None
        return f" skipped {count}"
    
    def reset(self):
        self.skipped_count = None
        
tiface = LoggingTransportInterface(tiface_inner)
tiface

# Filter layer setup

In [5]:
fiface = FilterInterface(tiface, FilterModeW("client"))

Find methods to run

In [6]:
[m for m in dir(fiface) if not m.startswith('__')]

['command_response', 'get_notifications', 'notif_poll_ms', 'run']

Run Filter!

In [7]:
run_fut = fiface.run()

# After waiting a bit, the above future should not have completed
time.sleep(0.1)
assert not run_fut.done()

LTI0: got transport notifications: []


Get the number of milliseconds between calling `get_notifications` on the contained `TransportInterface`. This can also be assigned to at any time. Lower values can cause performance issues, while higher values add latency for anything involving filter notifications.

In [8]:
fiface.notif_poll_ms

30

In [9]:
fiface.notif_poll_ms = 100

## Filter layer operation

### As a client, connect to server

In [10]:
import random
id = random.randint(1,10000)
connect_action = RequestActionW("connect", name="FilterNotebookClient{}".format(id), client_version="0.0.666")
connect_action

Connect { name: "FilterNotebookClient5958", client_version: "0.0.666" }

In [11]:
chococat_endpoint = EndpointW("157.230.134.224:2016")
filter_cmd = FilterCmdW("sendrequestaction", endpoint=chococat_endpoint, action=connect_action)
filter_cmd

SendRequestAction { endpoint: Endpoint(157.230.134.224:2016), action: Connect { name: "FilterNotebookClient5958", client_version: "0.0.666" } }

In [12]:
await fiface.command_response(filter_cmd)

LTI5 skipped 4: received transport_cmd SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 0, offset: 0 }, retry_interval: 250ms }], packets: [Request { sequence: 1, response_ack: None, cookie: None, action: Connect { name: "FilterNotebookClient5958", client_version: "0.0.666" } }] }
LTI6: got transport_rsp back from Transport layer: TransportRsp::EndpointError { error: Endpoint(157.230.134.224:2016) not found in transmit queue: Cannot update last_send }
LTI7: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Response { sequence: 0, request_ack: Some(0), code: LoggedIn { cookie: "qASkJnJMxu2hCk7r", server_version: "0.3.5" } } }]


Accepted

In [13]:
fiface.get_notifications()

[NewResponseCode { endpoint: Endpoint(157.230.134.224:2016), code: LoggedIn { cookie: "qASkJnJMxu2hCk7r", server_version: "0.3.5" } }]

In [14]:
new_response_code = _[0]
new_response_code

NewResponseCode { endpoint: Endpoint(157.230.134.224:2016), code: LoggedIn { cookie: "qASkJnJMxu2hCk7r", server_version: "0.3.5" } }

In [15]:
cookie = new_response_code.response_code.cookie
cookie

'qASkJnJMxu2hCk7r'

### Convenience function

In [16]:
async def request_action_to_filter(req_action, endpoint=chococat_endpoint):
    filter_cmd = FilterCmdW("sendrequestaction", endpoint=endpoint, action=req_action)
    return await fiface.command_response(filter_cmd)

### Join a room

In [17]:
await request_action_to_filter(RequestActionW("joinroom", room_name="general"))

LTI8: received transport_cmd SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 0 }, retry_interval: 250ms }], packets: [Request { sequence: 2, response_ack: Some(0), cookie: Some("qASkJnJMxu2hCk7r"), action: JoinRoom { room_name: "general" } }] }
LTI9: got transport_rsp back from Transport layer: TransportRsp::Accepted
LTI10: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Response { sequence: 1, request_ack: Some(1), code: JoinedRoom { room_name: "general" } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 5224126388335824814 } } }, TransportNotice::PacketDelivery { endpoint

Accepted

LTI11: received transport_cmd DropPacket { endpoint: Endpoint(157.230.134.224:2016), tid: ProcessUniqueId { prefix: 0, offset: 0 } }


Due to a bug in the Netwayste v1 server, there is a `BadRequest { error_msg: "cannot join game because in-game" }` filter notice following the `JoinedRoom` filter notice, below. Apparently, it's caused by a packet retry of the `JoinRoom` request action by the Transport layer, which is erroneously being treated as a second request action by the NWv1 server. It's erroneous because the request sequence is the same, so it should have been dropped by the server.

In [18]:
fiface.get_notifications()

[NewResponseCode { endpoint: Endpoint(157.230.134.224:2016), code: JoinedRoom { room_name: "general" } },
 NewChats { endpoint: Endpoint(157.230.134.224:2016), messages: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }] }]

LTI12: got transport_rsp back from Transport layer: TransportRsp::EndpointError { error: Transmit ID puid-0-0 not found for Endpoint(157.230.134.224:2016) in Transmit queue }
LTI13: received transport_cmd SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 1 }, retry_interval: 250ms }], packets: [UpdateReply { cookie: "", last_chat_seq: Some(12), last_game_update_seq: None, last_full_gen: None, partial_gen: None, pong: PingPong { nonce: 5224126388335824814 } }] }
LTI14: got transport_rsp back from Transport layer: TransportRsp::Accepted
LTI15: received transport_cmd DropPacket { endpoint: Endpoint(157.230.134.224:2016), tid: ProcessUniqueId { prefix: 1, offset: 1 } }
LTI16: got transport_rsp back from Transport layer: TransportRsp::Accepted
LTI17: received transport_cmd SendPackets { endpoint: Endpoint(157.230.134.224:2016), packet_infos: [PacketSettings { tid: ProcessUniqueId { prefix: 1, offset: 2 }, retry_

LTI60: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 10267332449114128788 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 2009136738199553529 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_up

LTI101: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 10853420374127335892 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 5146185462291750403 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_u

LTI148: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 9635115421837942460 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 11396763269595639101 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_u

...

LTI1972: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 609241775802752647 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 9432459909142925218 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_up

LTI2013: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 10299397668847831761 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 9312025905707846773 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_

LTI2054: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 15781849482019056157 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 17488350460284192715 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe

LTI2095: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 2911450184940337454 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 10714323816763551367 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_

LTI2136: got transport notifications: [TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 7934364864580194044 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_update: NoChange, ping: PingPong { nonce: 5581120194838368188 } } }, TransportNotice::PacketDelivery { endpoint: Endpoint(157.230.134.224:2016), packet: Update { chats: [BroadcastChatMessage { chat_seq: Some(12), player_name: "Server", message: "Player FilterNotebookClient5944 has left." }], game_update_seq: None, game_updates: [], universe_u

In [19]:
fiface.get_notifications()

[NewResponseCode { endpoint: Endpoint(157.230.134.224:2016), code: BadRequest { error_msg: "cannot join game because in-game" } }]